In [20]:
import pandas as pd

In [21]:
df = pd.read_csv("store_data.csv")
display(df.head(2))
display(df.shape)

,date,month,year,product_id,product_name,category,buy_price,sell_price,quantity,sales,profit_per_item,profit
0,2022-11-02,11,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,33.53,3,77.97,7.54,22.62
1,2022-11-02,11,2022,TEC-AC-10004144,"SanDisk Numeric Keypad, Bluetooth",Accessories,55.99,64.95,7,391.93,8.96,62.72


(10000, 12)

In [22]:
dflog = pd.read_csv("logistics.csv")
display(dflog.head(2))
display(dflog.shape)

,product_name,product_id,category,buy_price,sell_price,stock,profit_per_item
0,"Fellowes Folders, Blue",OFF-ST-10000988,Storage,25.99,33.53,16,7.54
1,"SanDisk Numeric Keypad, Bluetooth",TEC-AC-10004144,Accessories,55.99,64.95,10,8.96


(1856, 7)

In [23]:
dforder = dflog[["product_name","category","stock"]].head(0)
dforder

,product_name,category,stock


In [24]:
import streamlit as st
import snowflake.connector


### defining/resetting databases

In [25]:
# establishing the connection to snowflake
conn = snowflake.connector.connect(user = "mdimasaac", password = "Snowflake123", account = "vliobpf-yr09142")
# always begins with defining the cursor
cur = conn.cursor()
# create a new database
cur.execute("CREATE OR REPLACE DATABASE store_it")
# use that database
cur.execute("USE DATABASE store_it")
# create a new schema
cur.execute("CREATE OR REPLACE SCHEMA store_schema")
# use that schema
cur.execute("USE SCHEMA store_schema")
# create table 1: store_data
cur.execute('''CREATE OR REPLACE TABLE store_data (date VARCHAR, month VARCHAR, year INTEGER, product_id VARCHAR,
                product_name VARCHAR, category VARCHAR, buy_price FLOAT, sell_price FLOAT, quantity INTEGER,
                sales FLOAT, profit_per_item FLOAT, profit FLOAT)''')
data = df.values.tolist()
cur.executemany('''INSERT INTO store_data (date, month, year, product_id, product_name, category, buy_price, sell_price,
                    quantity, sales, profit_per_item, profit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''', data)
# create table 2: logistics
cur.execute('''CREATE OR REPLACE TABLE logistics (product_name VARCHAR, product_id VARCHAR, category VARCHAR, buy_price FLOAT,
                sell_price FLOAT, stock INTEGER, profit_per_item FLOAT)''')
datalog = dflog.values.tolist()
cur.executemany('''INSERT INTO logistics (product_name, product_id, category, buy_price, sell_price, stock, profit_per_item)
                VALUES (%s, %s, %s, %s, %s, %s, %s)''', datalog)
# create table 3: place order
cur.execute('''CREATE OR REPLACE TABLE place_order (date VARCHAR, product_name VARCHAR, category VARCHAR,
                stock INTEGER)''')
# datalog = dflog.values.tolist()
# cur.executemany('''INSERT INTO logistics (product_name, product_id, category, buy_price, sell_price, stock, profit_per_item)
#                 VALUES (%s, %s, %s, %s, %s, %s, %s)''', datalog)
# apply changes (?)
conn.commit()
# ends the action(s)
cur.close()
conn.close()

2023-05-05 23:42:08.679 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 3.0.2, Python Version: 3.8.0, Platform: Windows-10-10.0.19041-SP0
2023-05-05 23:42:08.680 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-05-05 23:42:09.458 INFO    snowflake.connector.cursor: query: [CREATE OR REPLACE DATABASE store_it]
2023-05-05 23:42:09.682 INFO    snowflake.connector.cursor: query execution done
2023-05-05 23:42:09.683 INFO    snowflake.connector.cursor: Number of results in first chunk: 1
2023-05-05 23:42:09.684 INFO    snowflake.connector.cursor: query: [USE DATABASE store_it]
2023-05-05 23:42:09.768 INFO    snowflake.connector.cursor: query execution done
2023-05-05 23:42:09.769 INFO    snowflake.connector.cursor: Number of res